In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit

In [ ]:
data = pd.DataFrame()
MAIN_URL = '' #Main URL -replaced.
URL = ''  # Source URL - replaced

In [ ]:
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    target_links = soup.find_all('a', href=True, hreflang='und')    
else:
    print(f'Failed to retrieve the page. Status code: {response.status_code}')

In [ ]:
for link in target_links:
        href = MAIN_URL + link['href']
        title = link.text
        data = data.append({'Link': href, 'Title': title}, ignore_index=True)

In [ ]:
def parse_text(url,data):
    text = str()
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')   
        target_elements = soup.find_all(class_=["field field--name-field-reading-text field--type-text-long field--label-hidden field--item"]) 
        if len(target_elements) > 0:
            text = target_elements[0].text
    else:
        print(f'Failed to retrieve the page. Status code: {response.status_code}')
    return text


In [ ]:
data['Text'] = 0
for i in range (len(data['Link'])):
    print(data['Link'][i])
    data['Text'][i] = parse_text(data['Link'][i],data)
    #print(parse_text(data['Link'][i],data))

In [ ]:
MODEL = f"cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

tf_model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
class_mapping = tf_model.config.id2label
for i in range(len(data['Text'])):
    text = data["Text"][i]
    print(text)
    tokens = tokenizer(text, return_tensors='tf')
    output = tf_model(**tokens)
    scores = output[0][0]
    scores = expit(scores)
    predictions = (scores >= 0.5) * 1
    # Map to classes
    for j in range(len(predictions)):
        if predictions[j]:
            data['Theme'][i] = class_mapping[j]